In [1]:
from finn.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe
import numpy as np

base_dir = "/workspace/finn/notebooks/dance-dance"
onnx_dir = f"{base_dir}/onnx"
verification_dir = f"{base_dir}/verification"

In [2]:
def compare(a, b):
    if np.isclose(a, b, atol=1e-3).all():
        print("Results are the same.")
    else:
        print("Results are not the same.")

### Load input

In [3]:
ibuf_normal = np.load(f"{verification_dir}/input.npy").astype(np.float32)
input_dict = {"global_in": ibuf_normal}

### Golden model

In [4]:
model = ModelWrapper(f"{onnx_dir}/dance-dance_tidy.onnx")

output_dict_golden = oxe.execute_onnx(model, input_dict)
output_golden = output_dict_golden['global_out']

### Python simulation

In [5]:
model = ModelWrapper(f"{onnx_dir}/dance-dance_streamlined.onnx")
                     
output_dict_python = oxe.execute_onnx(model, input_dict)
output_python = output_dict_python['global_out']
compare(output_python, output_golden)

Results are the same.


### C++ simulation

In [6]:
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model = ModelWrapper(f"{onnx_dir}/dance-dance_fold.onnx")

model = model.transform(GiveUniqueNodeNames())
model = model.transform(PrepareCppSim())
model = model.transform(CompileCppSim())
model = model.transform(SetExecMode("cppsim"))

model.save(f"{onnx_dir}/dance-dance_cppsim.onnx")

In [7]:
from finn.custom_op.registry import getCustomOp

parent_model = ModelWrapper(f"{onnx_dir}/dance-dance_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
child_model_filename = f"{onnx_dir}/dance-dance_cppsim.onnx"
getCustomOp(sdp_node).set_nodeattr("model", child_model_filename)

output_dict_cppsim = oxe.execute_onnx(parent_model, input_dict)
output_cppsim = output_dict_cppsim['global_out']
compare(output_cppsim, output_golden)

Results are the same.


### PyVerilator simulation

In [8]:
from finn.util.basic import pynq_part_map
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim

pynq_board = "Ultra96"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

child_model = ModelWrapper(f"{onnx_dir}/dance-dance_fold.onnx")

child_model = child_model.transform(GiveUniqueNodeNames())
child_model = child_model.transform(PrepareIP(fpga_part, target_clk_ns))
child_model = child_model.transform(HLSSynthIP())
child_model = child_model.transform(SetExecMode("rtlsim"))
child_model = child_model.transform(PrepareRTLSim())

child_model.save(f"{onnx_dir}/dance-dance_dataflow_child.onnx")

In [9]:
from finn.custom_op.registry import getCustomOp
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

parent_model = ModelWrapper(f"{onnx_dir}/dance-dance_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
sdp_node.set_nodeattr("model", f"{onnx_dir}/dance-dance_dataflow_child.onnx")
parent_model = parent_model.transform(SetExecMode("rtlsim"))

output_dict_verilator = oxe.execute_onnx(parent_model, input_dict)
output_verilator = output_dict_verilator['global_out']
compare(output_verilator, output_golden)

Results are the same.
